# Notebook for testing a simple FFNN

In [1]:
import autograd.numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn import datasets

In [2]:
from activation_functions import *
from cost_functions import *
from FFNN import *

In [3]:
network_input_size = 4
layer_output_sizes = [8, 5, 3]
activation_funcs = [sigmoid, ReLU, softmax]

In [4]:
def accuracy(predictions, targets):
    one_hot_predictions = np.zeros(predictions.shape)

    for i, prediction in enumerate(predictions):
        one_hot_predictions[i, np.argmax(prediction)] = 1
    return accuracy_score(one_hot_predictions, targets)

In [5]:
# Importing Iris data and functions from week 41
iris = datasets.load_iris()
inputs = iris.data

# Since each prediction is a vector with a score for each of the three types of flowers,
# we need to make each target a vector with a 1 for the correct flower and a 0 for the others.
targets = np.zeros((len(iris.data), 3))
for i, t in enumerate(iris.target):
    targets[i, t] = 1

In [6]:
NN = NeuralNetwork(network_input_size, layer_output_sizes, activation_funcs, mse)
NN_GDM = NeuralNetwork(network_input_size, layer_output_sizes, activation_funcs, mse)
NN_SGDM = NeuralNetwork(network_input_size, layer_output_sizes, activation_funcs, mse)

In [7]:
predictions = NN.predict(inputs)
predictions_GDM = NN_GDM.predict(inputs)
predictions_SGDM = NN_SGDM.predict(inputs)

print("Initial accuracy:")
print("GD:", accuracy(predictions, targets))
print("GDM:", accuracy(predictions_GDM, targets))
print("SGDM:", accuracy(predictions_SGDM, targets))

Initial accuracy:
GD: 0.6133333333333333
GDM: 0.4533333333333333
SGDM: 0.34


In [8]:
eta = 0.01
epochs = 100

for i in range(5):
    train_network(NN, inputs, targets, eta=eta, epochs=epochs)
    train_network_momentum(NN_GDM, inputs, targets, eta=eta, epochs=epochs)
    train_network_stochastic_momentum(NN_SGDM, inputs, targets, eta=eta, epochs=epochs)

    predictions = NN.predict(inputs)
    predictions_GDM = NN_GDM.predict(inputs)
    predictions_SGDM = NN_GDM.predict(inputs)

    print(f"Accuracy after {epochs*(i+1)} epochs:")
    print("GD:", accuracy(predictions, targets))
    print("GDM:", accuracy(predictions_GDM, targets))
    print("SGDM:", accuracy(predictions_SGDM, targets))

Accuracy after 100 epochs:
GD: 0.6333333333333333
GDM: 0.6133333333333333
SGDM: 0.6133333333333333
Accuracy after 200 epochs:
GD: 0.6466666666666666
GDM: 0.7666666666666667
SGDM: 0.7666666666666667
Accuracy after 300 epochs:
GD: 0.34
GDM: 0.8333333333333334
SGDM: 0.8333333333333334
Accuracy after 400 epochs:
GD: 0.34
GDM: 0.84
SGDM: 0.84
Accuracy after 500 epochs:
GD: 0.34
GDM: 0.88
SGDM: 0.88
